# 08 텍스트 분석
## 09 한글 텍스트 처리 - 네이버 영화 평점 감성 분석

#### KoNLPy 소개

+ KoNLPy는 파이썬의 대표적인 한글 형태소 패키지이다.
  + 형태소 : 단어로서 의미를 가지는 최소 단위
  + 헝태소 분석 (Morphological analysis) : 말뭉치를 형태소 어근 단위로 쪼개고 각 형태소에 품사 태깅(POS tagging)을 부착하는 작업

In [1]:
# ! pip install konlpy

#### 네이버 영화 평점 데이터를 기반으로 감성 분석 수행

__데이터 로딩__

https://github.com/e9t/nsmc 에서 네이버 영화 평점 데이터 다운로드

In [2]:
# 데이터 로딩

import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


학습 데이터 세트의 0과 1의 Label값 비율 살펴보기 (1이 긍정, 0이 부정 감성)

In [3]:
train_df['label'].value_counts( )

# 0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 나타내고 있다.

0    75173
1    74827
Name: label, dtype: int64

train_df의 경우 'document'칼럼에 Null이 일부 존재하므로 이 값을 공백으로 변환, 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 공백으로 변환

In [5]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# id 칼럼 삭제 수행
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)

+ 각 문장을 한글 형태소 분석을 통해 형태소 단어로 토큰화
    + 한글 형태소 엔진은 SNS 분석에 적합한 Twitter 클래스를 이용
    + Twitter 객체의 morphs() 메서드는 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체로 반환한다.

In [6]:
#  문장을 형태소 단어 형태로 반환하는 별도의 tokenizer 함수 생성
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

c:\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


+ 사이킷런의 TfidVectorizer를 이용해 TF-IDF 피처 모델 생성
    + ngramg은 (1,2)
    + min_df=3
    + max_df는 상위 90%로 제한

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

c:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


KeyboardInterrupt: 

+ 로지스틱 회귀를 이용해 분류 기반의 감성 분석 수행
  + 로지스틱 회귀의 하이퍼 파라미터 C의 최적화를 위해 GridSearchCV 이용

In [ ]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0, solver='liblinear')

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

C가 3.5일 때 최고 0.8593의 정확도를 보인다.

+ 테스트 세트를 이용해 최종 감성 분석 예측 수행
  + 테스트 세트를 이용해 예측할 때는 학습할 때 적용한 TfidVectorizer를 그대로 사용해야 함
  + 그래야만 학습 시 설정된 TfidVectorizer의 피처 개수와 테스트 데이터를 TfidVectorizer로 변환할 피처 개수가 같아진다.
  + 학습 데이터에 사용된 TfidVectorizer 객체 변수인 tfid_Vect를 이용해 transform()을 테스트 데이터의 document 칼럼에 수행한다.

In [ ]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))